# Exports packet loss data for ML outside of jupyter

### import all the packages needed for this task

In [1]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions, helpers
from elasticsearch.helpers import scan
import sys
import datetime
import pandas as pd

### Select time range

In [2]:
cdt = datetime.datetime.utcnow()
#cdt = datetime.datetime(2017,1,21,9,0,0)

GT = (cdt - datetime.timedelta(hours=30*24)).strftime("%Y%m%dT%H%m%S+0000")
LT = cdt.strftime("%Y%m%dT%H%m%S+0000")
print('between: ', GT, ' and ', LT)

between:  20170313T200325+0000  and  20170412T200425+0000


### establish the Elastic Search connection

In [3]:
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

### list all network\_weather* indices

In [4]:
indices = es.cat.indices(index="network_weather-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

### indices to be used

In [5]:
ind=['network_weather-2017.3.*','network_weather-2017.4.*']
print('will use indices:', ind)

will use indices: ['network_weather-2017.3.*', 'network_weather-2017.4.*']


In [35]:
query={
   "size": 0,
   "_source": ["src", "dest", "packet_loss","timestamp"],
   "query": {
        "bool": {
          "must": [
            {"term": { "_type" : "packet_loss_rate"}},
            {"term": { "srcProduction" : True }},
            {"term": { "destProduction" : True }},
            {"term": { "dest" : '149.165.225.223' }},
          ],
          "filter" : {
            "range" : {
              "timestamp" :{ "gt" : GT, "lt" : LT }
            }
          }
        }
    }
}


data=[]
res = scan(es, query=query, index=ind, scroll='5m', timeout="5m", size=1000)

count = 0
for pr in res:
    r=pr['_source']
    ts=datetime.datetime.strptime(r['timestamp'], "%Y%m%dT%H%M%S+0000").timestamp()
    doc=[ts,r['src'],r['dest'],r['packet_loss']]
    data.append(doc)
    if not count%1000000:
        print(count)
    #if count>10: break
    #print(doc)
    count+=1
    
packet_loss=pd.DataFrame(data,columns=['timestamp', 'src', 'dest', 'packet_loss'])
del data
#print(res)

0


In [36]:
packet_loss.to_csv('packet_loss_iu_dest.csv',index=False) #,compression='gzip')